# Recommender for MovieLens

### Getting MovieLens data

* Download the movielens 100k dataset from this link: [ml-100k.zip](http://files.grouplens.org/datasets/movielens/ml-100k.zip)

* Upload ml-100k.zip

* Extract using the following cell:

In [1]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=fda410871205d60a981c0db320cc72a6625f85a4a5c61e7e8526dbfb8bd0e189
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [0]:
# import required libraries
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import pairwise_distances
from heapq import nlargest
from sklearn.metrics import mean_squared_error
from math import sqrt
import os.path
import wget

In [3]:
# define constant for movie lends 100K directory
wget.download("https://github.com/MIE451-1513-2019/course-datasets/raw/master/ml-100k.zip")
!unzip ml-100k.zip
MOVIELENS_DIR = "ml-100k/"

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


### Building the recommender

## Loading the data

First, we inspect the directory content

In [4]:
!ls $MOVIELENS_DIR

allbut.pl  u1.base  u2.test  u4.base  u5.test  ub.base	u.genre  u.occupation
mku.sh	   u1.test  u3.base  u4.test  ua.base  ub.test	u.info	 u.user
README	   u2.base  u3.test  u5.base  ua.test  u.data	u.item


We then load the full MovieLens 100K dataset to find the number of users and items

In [5]:
fields = ['userID', 'itemID', 'rating', 'timestamp']
ratingDF = pd.read_csv(os.path.join(MOVIELENS_DIR, 'u.data'), sep='\t', names=fields)

ratingDF.head()

,userID,itemID,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
numUsers = len(ratingDF.userID.unique())
numItems = len(ratingDF.itemID.unique())

print("Number of users:", numUsers)
print("Number of items:", numItems)

Number of users: 943
Number of items: 1682


In [7]:
fieldsMovies = ['movieID', 'movieTitle', 'releaseDate', 'videoReleaseDate', 'IMDbURL', 'unknown', 'action', 'adventure',
          'animation', 'childrens', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'filmNoir', 'horror',
          'musical', 'mystery', 'romance','sciFi', 'thriller', 'war', 'western']
moviesDF = pd.read_csv(os.path.join(MOVIELENS_DIR, 'u.item'), sep='|', names=fieldsMovies, encoding='latin-1')

moviesDF.head()

,movieID,movieTitle,releaseDate,videoReleaseDate,IMDbURL,unknown,action,adventure,animation,childrens,comedy,crime,documentary,drama,fantasy,filmNoir,horror,musical,mystery,romance,sciFi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


Then, we load a train-test split

In [8]:
trainDF = pd.read_csv(os.path.join(MOVIELENS_DIR, 'u1.base'), sep='\t', names=fields)
testDF = pd.read_csv(os.path.join(MOVIELENS_DIR, 'u1.test'), sep='\t', names=fields)

# test number of records (total should be 100K)
print("# of lines in train:", trainDF.shape[0])
print("# of lines in test:", testDF.shape[0])

# of lines in train: 80000
# of lines in test: 20000


## Building User-to-Item Rating Matrix

In [0]:
def buildUserItemMatrix(dataset, numUsers, numItems):
    # Initialize a of size (numUsers, numItems) to zeros
    matrix = np.zeros((numUsers, numItems), dtype=np.int8)
    
    # Populate the matrix based on the dataset
    for (index, userID, itemID, rating, timestamp) in dataset.itertuples():
        matrix[userID-1, itemID-1] = rating
    return matrix

In [0]:
trainUserItemMatrix = buildUserItemMatrix(trainDF, numUsers, numItems)
testUserItemMatrix = buildUserItemMatrix(testDF, numUsers, numItems)

In [11]:
# Inspect the train matrix
# What's the shape of this matrix
# Let's say the number of users is U and the number of items is I, I will ask you the dimensions of some matrixes in this lab
trainUserItemMatrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]], dtype=int8)

## Baseline solution - Average User Rating

In [0]:
def predictByUserAverage(trainSet, numUsers, numItems):
    # Initialize the predicted rating matrix with zeros
    predictionMatrix = np.zeros((numUsers, numItems))
    
    for (user,item), rating in np.ndenumerate(trainSet):
        # Predict rating for every item that wasn't ranked by the user (rating == 0)
        if rating == 0:
            # select the row for user
            # what's the shape of userVector
            userVector = trainSet[user, :]
            
            # Extract the items the user already rated
            ratedItems = userVector[userVector.nonzero()]
            
            # If not empty, calculate average and set as rating for the current item
            if ratedItems.size == 0:
                itemAvg = 0
            else:
                itemAvg = ratedItems.mean()
            predictionMatrix[user, item] = itemAvg
            
        # report progress every 100 users
        if (user % 100 == 0 and item == 1):
            print ("calculated %d users" % (user,))
    
    return predictionMatrix

In [13]:
userAvgPreiction = predictByUserAverage(trainUserItemMatrix, numUsers, numItems)

calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


## How well did we do?

In [0]:
def rmse(pred, test):
    # calculate RMSE for all the items in the test dataset
    predItems = pred[test.nonzero()].flatten() 
    testItems = test[test.nonzero()].flatten()
    return sqrt(mean_squared_error(predItems, testItems))

In [15]:
rmse(userAvgPreiction, testUserItemMatrix)

1.0629951276561334

## User-User Similarity

### Question: why do we divide the result by the sume of similaries?

In [0]:
userSimilarity = 1 - pairwise_distances(trainUserItemMatrix, metric='cosine')

In [17]:
userSimilarity

array([[1.        , 0.09702087, 0.05246924, ..., 0.15815144, 0.13309031,
        0.25277792],
       [0.09702087, 1.        , 0.05134808, ..., 0.13280638, 0.1265973 ,
        0.10178363],
       [0.05246924, 0.05134808, 1.        , ..., 0.08353707, 0.08122974,
        0.01967584],
       ...,
       [0.15815144, 0.13280638, 0.08353707, ..., 1.        , 0.1016418 ,
        0.09511958],
       [0.13309031, 0.1265973 , 0.08122974, ..., 0.1016418 , 1.        ,
        0.18246466],
       [0.25277792, 0.10178363, 0.01967584, ..., 0.09511958, 0.18246466,
        1.        ]])

In [0]:
def kNearestNeighbor(sim, k):
    sim = sim.copy()
    m,n = sim.shape
    for i in range(m):
        # only keep the k largest number
        sim[i, sim[i].argsort()[:-k]] = 0
    return sim

In [0]:
userSimilarity=kNearestNeighbor(userSimilarity, 3)

In [20]:
np.count_nonzero(userSimilarity[0])

3

In [0]:
def predictByUserSimilarity(trainSet, numUsers, numItems, similarity, k=None):
    
    if k is not None:
        similarity = kNearestNeighbor(similarity, k)
    
    # Initialize the predicted rating matrix with zeros
    predictionMatrix = np.zeros((numUsers, numItems))
    
    for (user,item), rating in np.ndenumerate(trainSet):
        
        # Predict rating for every item that wasn't ranked by the user (rating == 0)
        if rating == 0:
            # Extract the users that provided rating for this item
            # shape of itemVector
            itemVector = trainSet[:,item]
            usersRatings = itemVector[itemVector.nonzero()]

            # Get the similarity score for each of the users that provided rating for this item
            # what's the shape of similarity, similarity[user,:]
            usersSim = similarity[user,:][itemVector.nonzero()]

            # If there no users that ranked this item, use user's average
            if len(usersSim) == 0:
                userVector = trainSet[user, :]
                ratedItems = userVector[userVector.nonzero()]
                
                # If the user didnt rated any item use 0, otherwise use average
                if len(ratedItems) == 0:
                    predictionMatrix[user,item] = 0
                else:
                    predictionMatrix[user,item] = ratedItems.mean()
            else:
                # predict score based on user-user similarity
                predictionMatrix[user,item] = (usersRatings*usersSim).sum() / usersSim.sum()

        if not np.isfinite(predictionMatrix[user,item]):
            predictionMatrix[user,item]=0
        
        # report progress every 100 users
        if (user % 100 == 0 and item == 1):
            print ("calculated %d users" % (user,))
    
    return predictionMatrix

In [22]:
userSimPrediction = predictByUserSimilarity(trainUserItemMatrix, numUsers, numItems, userSimilarity, k=50)

calculated 0 users


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars


calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


In [23]:
rmse(userSimPrediction, testUserItemMatrix)

2.3177794082696836

## Vectorized Version

In [0]:
def vectorizedUserSimRecSys(trainSet, k=None):
    # Initialize the predicted rating matrix with zeros
    temp_matrix = np.zeros(trainSet.shape)
    temp_matrix[trainSet.nonzero()] = 1
    uu_similarity = 1 - pairwise_distances(trainSet, metric='cosine')
    if k is not None:
        uu_similarity = kNearestNeighbor(uu_similarity, k)
    # UxI: UxU mul UxI
    normalizer = np.matmul(uu_similarity, temp_matrix)
    #print(normalizer)
    normalizer[normalizer == 0] = 1e-5
    #what's the dimension of np.matmul(uu_similarity, trainSet)
    
    predictionMatrix = np.matmul(uu_similarity, trainSet)/normalizer
    #predictionMatrix[temp_matrix.nonzero()] = 0
    #Cold start
    # if no one has rated this item before, use user average  
    useraverage = np.sum(trainSet, axis=1)/np.sum(temp_matrix, axis=1)
    columns = np.sum(predictionMatrix, axis=0)
    #print(columns.shape)
    predictionMatrix[:, columns==0] = predictionMatrix[:, columns==0] + np.expand_dims(useraverage, axis=1)
    
    return predictionMatrix

In [25]:
userSimPrediction = vectorizedUserSimRecSys(trainUserItemMatrix, k=50)
print(userSimPrediction)

[[3.95768482 3.07884915 2.95870738 ... 0.         3.         3.        ]
 [3.84711903 0.         2.00999946 ... 0.         0.         0.        ]
 [4.         0.         3.         ... 2.         0.         0.        ]
 ...
 [3.89075627 0.         3.33135748 ... 0.         0.         0.        ]
 [3.9933765  3.66514556 2.65346589 ... 0.         0.         0.        ]
 [4.07279746 3.40602568 3.42834447 ... 0.         3.         3.        ]]


In [26]:
rmse(userSimPrediction, testUserItemMatrix)

1.1143635272661785

## Precision@k and Recall@k

### Question: where does the K come from?

In [0]:
def avgPrecisionAtK(testSet, prediction, k):
    # Initialize sum and count vars for average calculation
    sumPrecisions = 0
    countPrecisions = 0
    
    # Define function for converting 1-5 rating to 0/1 (like / don't like)
    vf = np.vectorize(lambda x: 1 if x >= 4 else 0)
    
    for userID in range(numUsers):
        # Pick top K based on predicted rating
        userVector = prediction[userID,:]
        #print(userVector)
        topK = nlargest(k, range(len(userVector)), userVector.take)
        #print(topK)
        # Convert test set ratings to like / don't like
        userTestVector = vf(testSet[userID,:]).nonzero()[0]
        #print(vf(testSet[userID,:]).nonzero())
        # Ignore user if has no ratings in the test set
        if (len(userTestVector) == 0):
            continue

        # Calculate precision
        precision = float(len([item for item in topK if item in userTestVector]))/len(topK)

        # Update sum and count
        sumPrecisions += precision
        countPrecisions += 1
        
    # Return average P@k
    return float(sumPrecisions)/countPrecisions

In [0]:
def avgRecallAtK(testSet, prediction, k):
    # Initialize sum and count vars for average calculation
    sumRecalls = 0
    countRecalls = 0
    
    # Define function for converting 1-5 rating to 0/1 (like / don't like)
    vf = np.vectorize(lambda x: 1 if x >= 4 else 0)
    
    for userID in range(numUsers):
        # Pick top K based on predicted rating
        userVector = prediction[userID,:]
        topK = nlargest(k, range(len(userVector)), userVector.take)
        
        # Convert test set ratings to like / don't like
        userTestVector = vf(testSet[userID,:]).nonzero()[0]
        
        # Ignore user if has no ratings in the test set
        if (len(userTestVector) == 0):
            continue
        
        # Calculate recall
        recall = float(len([item for item in topK if item in userTestVector]))/len(userTestVector)
        
        # Update sum and count
        sumRecalls += recall
        countRecalls += 1
    
    # Return average R@k
    return float(sumRecalls)/countRecalls

In [29]:
print("k\tP@k\tR@k")
for k in [25, 50, 100, 250, 500]:
    print("%d\t%.3lf\t%.3lf" % (k, avgPrecisionAtK(testUserItemMatrix, userSimPrediction, k), avgRecallAtK(testUserItemMatrix, userSimPrediction, k)))

k	P@k	R@k
25	0.018	0.026
50	0.037	0.089
100	0.050	0.249
250	0.043	0.546
500	0.038	0.847


In [30]:
userSimPrediction

array([[3.95768482, 3.07884915, 2.95870738, ..., 0.        , 3.        ,
        3.        ],
       [3.84711903, 0.        , 2.00999946, ..., 0.        , 0.        ,
        0.        ],
       [4.        , 0.        , 3.        , ..., 2.        , 0.        ,
        0.        ],
       ...,
       [3.89075627, 0.        , 3.33135748, ..., 0.        , 0.        ,
        0.        ],
       [3.9933765 , 3.66514556, 2.65346589, ..., 0.        , 0.        ,
        0.        ],
       [4.07279746, 3.40602568, 3.42834447, ..., 0.        , 3.        ,
        3.        ]])

In [31]:
testUserItemMatrix[0]

array([0, 0, 0, ..., 0, 0, 0], dtype=int8)

## Popularity Based Recommendations

In [0]:
def predictByPopularity(trainSet, numUsers, numItems):
    # Initialize the predicted rating matrix with zeros
    predictionMatrix = np.zeros((numUsers, numItems))
    
    # Define function for converting 1-5 rating to 0/1 (like / don't like)
    vf = np.vectorize(lambda x: 1 if x >= 4 else 0)
    
    # For every item calculate the number of people liked (4-5) divided by the number of people that rated
    itemPopularity = np.zeros((numItems))
    for item in range(numItems):
        numOfUsersRated = len(trainSet[:, item].nonzero()[0])
        numOfUsersLiked = len(vf(trainSet[:, item]).nonzero()[0])
        if numOfUsersRated == 0:
            itemPopularity[item] = 0
        else:
            itemPopularity[item] = numOfUsersLiked/numOfUsersRated
    
    for (user,item), rating in np.ndenumerate(trainSet):
        # Predict rating for every item that wasn't ranked by the user (rating == 0)
        if rating == 0:
            predictionMatrix[user, item] = itemPopularity[item]
            
        # report progress every 100 users
        if (user % 100 == 0 and item == 1):
            print ("calculated %d users" % (user,))
    
    return predictionMatrix

In [33]:
popPreiction = predictByPopularity(trainUserItemMatrix, numUsers, numItems)

calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


In [34]:
print("k\tP@k\tR@k")
for k in [25, 50, 100, 250, 500]:
    print("%d\t%.3lf\t%.3lf" % (k, avgPrecisionAtK(testUserItemMatrix, popPreiction, k), avgRecallAtK(testUserItemMatrix, popPreiction, k)))

k	P@k	R@k
25	0.001	0.000
50	0.009	0.015
100	0.039	0.155
250	0.043	0.400
500	0.036	0.688


## Making Recommendations for a User

In [0]:
def userTopK(prediction, moviesDataset, userID, k):
    # Pick top K based on predicted rating
    userVector = prediction[userID+1,:]
    topK = nlargest(k, range(len(userVector)), userVector.take)
    namesTopK = list(map(lambda x: moviesDataset[moviesDataset.movieID == x+1]["movieTitle"].values[0], topK))
    return namesTopK

In [36]:
# recommend for userID 350 according to popularity recommender
userTopK(popPreiction, moviesDF, 350, 10)

['Loch Ness (1995)',
 'Perfect Candidate, A (1996)',
 'Love and Death on Long Island (1997)',
 'Crossfire (1947)',
 'Celestial Clockwork (1994)',
 'They Made Me a Criminal (1939)',
 'Last Time I Saw Paris, The (1954)',
 'Innocents, The (1961)',
 "Jupiter's Wife (1994)",
 'Prefontaine (1997)']

In [37]:
# recommend for userID 350 according to average rating recommender
userTopK(userAvgPreiction, moviesDF, 350, 10)

['Toy Story (1995)',
 'GoldenEye (1995)',
 'Four Rooms (1995)',
 'Get Shorty (1995)',
 'Copycat (1995)',
 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)',
 'Twelve Monkeys (1995)',
 'Babe (1995)',
 'Dead Man Walking (1995)',
 'Richard III (1995)']

In [38]:
# recommend for userID 350 according to user similarity recommender
userTopK(userSimPrediction, moviesDF, 350, 10)

['Three Colors: Red (1994)',
 'Spitfire Grill, The (1996)',
 'Mirror Has Two Faces, The (1996)',
 'Secrets & Lies (1996)',
 'Apostle, The (1997)',
 'Bananas (1971)',
 'Candidate, The (1972)',
 'Great Dictator, The (1940)',
 'Rebecca (1940)',
 'Laura (1944)']

## Evaluating the other datasets

In [0]:
datasetsFileNames = [('u2.base', 'u2.test'),
                     ('u3.base', 'u3.test'),
                     ('u4.base', 'u4.test'),
                     ('u5.base', 'u5.test')]

In [40]:
rmseList = []
for trainFileName, testFileName in datasetsFileNames:
    curTrainDF = pd.read_csv(os.path.join(MOVIELENS_DIR, trainFileName), sep='\t', names=fields)
    curTestDF = pd.read_csv(os.path.join(MOVIELENS_DIR, testFileName), sep='\t', names=fields)
    curTrainUserItemMatrix = buildUserItemMatrix(curTrainDF, numUsers, numItems)
    curTestUserItemMatrix = buildUserItemMatrix(curTestDF, numUsers, numItems)
    #prediction with average
    curUserAvgPreiction = predictByUserAverage(curTrainUserItemMatrix, numUsers, numItems)
    avgRMSE = rmse(curUserAvgPreiction, curTestUserItemMatrix)
    #prediction with similarity
    curUserSimilarity = 1 - pairwise_distances(curTrainUserItemMatrix, metric='cosine')
    curUserSimPreiction = predictByUserSimilarity(curTrainUserItemMatrix, numUsers, numItems, curUserSimilarity)
    simRMSE = rmse(curUserSimPreiction, curTestUserItemMatrix)
    
    rmseList.append((avgRMSE, simRMSE))

calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users
calculated 0 users


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars


calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users
calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users
calculated 0 users


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars


calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users
calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users
calculated 0 users


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars


calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users
calculated 0 users
calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users
calculated 0 users


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars


calculated 100 users
calculated 200 users
calculated 300 users
calculated 400 users
calculated 500 users
calculated 600 users
calculated 700 users
calculated 800 users
calculated 900 users


In [41]:
print("Avg\tSim")
for avgScore, simScore in rmseList:
    print("%.3lf\t%.3lf" % (avgScore, simScore))

Avg	Sim
1.047	1.021
1.033	1.013
1.037	1.009
1.039	1.016
